In [2]:
import warnings, logging
warnings.filterwarnings("ignore",
                        category=UserWarning,
                        module=r"rpy2\.rinterface_lib")
warnings.filterwarnings("ignore",
                        category=UserWarning,
                        module=r"rpy2\.rinterface")

import rpy2.robjects as ro
from   rpy2.robjects.packages import importr
import rpy2.rinterface_lib.callbacks as cb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re


cb.logger.setLevel(logging.ERROR)
base      = importr("base")

In [3]:

# ------------------------------------------------------------------
# Mandar a R el codigo
# ------------------------------------------------------------------
ro.r("""
suppressPackageStartupMessages({
  library(simona)
  library(igraph)
  library(dplyr)
  setwd("~/Escritorio/SimPython")

  # cargar RDF
  rdf_data1 <- read.csv("cie10.ttl", stringsAsFactors = FALSE)
  rdf_data2 <- read.csv("ciap.ttl", stringsAsFactors = FALSE)
  rdf_data3 <- read.csv("snomed.ttl", stringsAsFactors = FALSE)
  rdf_mapeos <- read.csv("mapeosCiapSnomedCie10.ttl", stringsAsFactors = FALSE)

  # construir igraph con los RDF
  g1 <- graph_from_data_frame(d = rdf_data1[, c("Sujeto", "Objeto")], directed = TRUE)
  E(g1)$label <- rdf_data1$Predicado
  g2 <- graph_from_data_frame(d = rdf_data2[, c("Sujeto", "Objeto")], directed = TRUE)
  E(g2)$label <- rdf_data2$Predicado
  g3 <- graph_from_data_frame(d = rdf_data3[, c("Sujeto", "Objeto")], directed = TRUE)
  E(g3)$label <- rdf_data3$Predicado 
  nodos_cie10 <- names(V(g1))
  nodos_ciap1 <- names(V(g2))
  nodos_snomed <- names(V(g3))

  # crear DAG con el grafo
  ontology1 <- create_ontology_DAG_from_igraph(g1, E(g1)$label)
  ontology2 <- create_ontology_DAG_from_igraph(g2, E(g2)$label)
  ontology3 <- create_ontology_DAG_from_igraph(g3, E(g3)$label)

  # Función de agregación
  get_max_similarity <- function(term_a, term_b, mapeos_df, g1, g2, g3, ont_cie10, ont_snomed, sim_method) {
     
    if (term_a == term_b) {
     return(1.0)
    }
    
    # auxiliar para determinar la similitud con simona
    sim_ont <- function(term1, term2, ontologia) {
      if (term1 == term2) return(1.0)
      tryCatch({
        if (sim_method == "Sim_Wang_2007") {
          sim_matrix <- term_sim(ontologia, terms = c(term1, term2), method = sim_method,
                                control = list(contribution_factor = c("contains" = 0.75, "include" = 0.8, "hasFindingSite" = 0.86, "laterality" = 0.87, "Isa" = 0.85, "synonym"= 0.9)))
        } else {
          sim_matrix <- term_sim(ontologia, terms = c(term1, term2), method = sim_method)
        }
        if (is.null(sim_matrix) || !is.matrix(sim_matrix) || !all(dim(sim_matrix) == c(2, 2))) return(NA)
        sim_matrix[1, 2]
      }, error = function(e) {
        message("Error al calcular term_sim: ", e$message)
        return(NA)
      })
    }
    
    # auxiliar para obtener mapeos
    get_mapeos <- function(term, target, predicado) {
      mapeos_df %>%
        filter(Sujeto == term, Predicado == predicado) %>%
        pull(Objeto) %>%
        unique()
    }
    
    
    nodos_cie10 <- names(V(g1))
    nodos_ciap1 <- names(V(g2))
    nodos_snomed <- names(V(g3))
    
    # CASO 1: CIAP - CIE10
    if ((term_a %in% nodos_ciap1 && term_b %in% nodos_cie10) || (term_b %in% nodos_ciap1 && term_a %in% nodos_cie10)) {
      if (term_a %in% nodos_ciap1) { term_ciap <- term_a; term_cie <- term_b }
      else { term_ciap <- term_b; term_cie <- term_a }
      mapeos <- get_mapeos(term_ciap, nodos_cie10, "hasICD10")
      
      if (length(mapeos) > 0) {
        similitudes <- sapply(mapeos, function(x) sim_ont(x, term_cie, ont_cie10)) * 0.95
      } else {
        # fallback a snomed
        mapeos_ciap_sno <- get_mapeos(term_ciap, nodos_snomed, "hasSNOMED")
        mapeos_cie_sno <- get_mapeos(term_cie, nodos_snomed, "hasSNOMED")
        pares <- expand.grid(mapeos_ciap_sno, mapeos_cie_sno, stringsAsFactors = FALSE)
        similitudes <- apply(pares, 1, function(par) sim_ont(par[1], par[2], ont_snomed)) * 0.95
      }
      
      # CASO 2: CIAP - SNOMED (parecido)
    } else if ((term_a %in% nodos_ciap1 && term_b %in% nodos_snomed) || (term_b %in% nodos_ciap1 && term_a %in% nodos_snomed)) {
      if (term_a %in% nodos_ciap1) { term_ciap <- term_a; term_sno <- term_b }
      else { term_ciap <- term_b; term_sno <- term_a }
      
      mapeos <- get_mapeos(term_ciap, nodos_snomed, "hasSNOMED")
      
      if (length(mapeos) > 0) {
        similitudes <- sapply(mapeos, function(x) sim_ont(x, term_sno, ont_snomed)) * 0.95
      } else {
        # fallback a CIE-10 (nunca ocurre)
        mapeos_ciap_cie <- get_mapeos(term_ciap, nodos_cie10, "hasICD10")
        mapeos_sno_cie <- get_mapeos(term_sno, nodos_cie10, "hasICD10")
        pares <- expand.grid(mapeos_ciap_cie, mapeos_sno_cie, stringsAsFactors = FALSE)
        similitudes <- apply(pares, 1, function(par) sim_ont(par[1], par[2], ont_cie10)) * 0.95
      }
      
      # CASO 3: CIAP - CIAP
    } else if (term_a %in% nodos_ciap1 && term_b %in% nodos_ciap1) {
      term_ciap1 <- term_a; term_ciap2 <- term_b
      
      mapeos1_cie <- get_mapeos(term_ciap1, nodos_cie10, "hasICD10")
      mapeos2_cie <- get_mapeos(term_ciap2, nodos_cie10, "hasICD10")
      mapeos1_sno <- get_mapeos(term_ciap1, nodos_snomed, "hasSNOMED")
      mapeos2_sno <- get_mapeos(term_ciap2, nodos_snomed, "hasSNOMED")
      
      pares_cie <- expand.grid(mapeos1_cie, mapeos2_cie, stringsAsFactors = FALSE)
      pares_sno <- expand.grid(mapeos1_sno, mapeos2_sno, stringsAsFactors = FALSE)
      
      similitudes_cie <- apply(pares_cie, 1, function(par) sim_ont(par[1], par[2], ont_cie10))
      similitudes_sno <- apply(pares_sno, 1, function(par) sim_ont(par[1], par[2], ont_snomed))
      
      similitudes <- c(similitudes_cie, similitudes_sno) * 0.95
    
      # CASO 4: SNOMED - SNOMED
    } else if (term_a %in% nodos_snomed && term_b %in% nodos_snomed) {
      similitud <- sim_ont(term_a, term_b, ont_snomed)
      similitudes <- similitud
     
      # CASO 5: CIE10 - CIE10
    } else if (term_a %in% nodos_cie10 && term_b %in% nodos_cie10) {
      similitud <- sim_ont(term_a, term_b, ont_cie10)
      similitudes <- similitud

    } else {
      warning("No se pudo determinar un término.")
      return(NA)
    }
    
    similitudes_validas <- similitudes[!is.na(similitudes)]
    if (length(similitudes_validas) == 0) return(0)
    
    # agregar usando max (mapeos son 0.95)
    sim_final <- max(similitudes_validas)
    return(sim_final)
    
  }
      
  get_sim_R <- function(code1, code2,
                      metodo = "Sim_Wang_2007") {
      get_max_similarity(code1, code2,
                          rdf_mapeos,
                          g1, g2, g3,
                          ontology1, ontology3,
                          metodo)
  }

  get_mapeos<- function(term, target, predicado) {
    rdf_mapeos %>%
      filter(Sujeto == term, Predicado == predicado) %>%
      pull(Objeto) %>%
      unique()
  }
     
  get_mapeos_R <- function(term, target_vocab) {
    if (target_vocab == "CIAP1") {
      target_nodes <- nodos_ciap1
      predicado <- "hasCIAP1"
    } else if (target_vocab == "CIE10") {
      target_nodes <- nodos_cie10
      predicado <- "hasICD10"
    } else if (target_vocab == "SNOMED") {
      target_nodes <- nodos_snomed
      predicado <- "hasSNOMED"
    } else {
      warning("Vocabulario desconocido.")
      return(character(0))
    }
    
    get_mapeos(term, target_nodes, predicado)
  }

  get_top10_codes_mapeo_vecinos <- function(source_code, target_vocab, max_results = 10) {
    # grafo según el código origen
    if (source_code %in% names(V(g1))) {
      g_actual <- g1
    } else if (source_code %in% names(V(g2))) {
      g_actual <- g2
    } else if (source_code %in% names(V(g3))) {
      g_actual <- g3
    } else {
      warning("Código no encontrado en ninguno de los grafos.")
      return(character(0))
    }
    
    visited <- character(0)
    result_set <- character(0)
    queue <- source_code
    queue <- as.character(queue)
    
    while (length(queue) > 0 && length(result_set) < max_results) {
      queue <- as.character(queue)
      current_code <- queue[1]
      queue <- queue[-1]
      
      if (current_code %in% visited) next
      visited <- c(visited, current_code)
      
      # mapeos del código actual usando rdf_mapeos
      mapeos_actuales <- get_mapeos_R(current_code, target_vocab)
      nuevos_mapeos <- setdiff(mapeos_actuales, result_set)
      
      if (length(nuevos_mapeos) > 0) {
        result_set <- unique(c(result_set, nuevos_mapeos))
      }
      
      if (length(result_set) >= max_results) break
      
      # vecinos en el grafo actual
      vecinos <- names(neighbors(g_actual, current_code, mode = "all"))
      vecinos <- as.character(vecinos)
      vecinos <- setdiff(vecinos, visited)
      queue <- c(queue, vecinos)
    }
    
    return(head(result_set, max_results))
  }
     
})
""")

get_sim = ro.globalenv['get_sim_R']
get_map = ro.globalenv['get_mapeos_R']
get_top10 = ro.globalenv['get_top10_codes_mapeo_vecinos']

In [22]:
# ------------------------------------------------------------------
# Llamada desde Python
# ------------------------------------------------------------------

# Comparar code_gt y diagCD de los csv
df1 = pd.read_csv("datasetEtiquetado_INTER_1.csv")  
df2 = pd.read_csv("bioLordCiap_toCompare_withGrafo.csv")

merged_df = pd.merge(df1, df2, on="indice")
merged_df = merged_df.dropna(subset=["code_gt", "diagCD"])

resultados = []
for _, row in merged_df.iterrows():
    code_gt = str(row["code_gt"]).strip()
    diagCD = str(row["diagCD"]).strip()
    desc = str(row["diag_st"]).strip()
    idx = row["indice"]

    if code_gt.upper() == "NO CODIFICABLE":
        continue

    codes_gt = [c.strip() for c in code_gt.split(";") if c.strip()]
    similitudes = []
    for code in codes_gt:
        try:
            sim = get_sim(code, diagCD)
            sim_val = float(sim[0])
            similitudes.append(f"{sim_val:.4f}")
            print(f"Similitud fila {idx} con códigos {code} vs {diagCD}: {sim_val}")
        except Exception as e:
            print(f"Error en fila {idx} con códigos {code} vs {diagCD}: {e}")
            sim_val = None
            similitudes.append("NA")

    sim_val_str = ";".join(similitudes)
    
    resultados.append({
        "indice": idx,
        "diag_st": desc,
        "code_gt": code_gt,
        "diagCD": diagCD,
        "similitud": sim_val_str
    })

result_df = pd.DataFrame(resultados)
result_df.to_csv("resultados_similitud.csv", index=False)
print("Resultados guardados")

Similitud fila 3 con códigos A10 vs A10: 1.0
Similitud fila 9 con códigos B02 vs B02: 1.0
Similitud fila 10 con códigos S74 vs S88: 0.580174466325389
Similitud fila 10 con códigos S88 vs S88: 1.0
Similitud fila 11 con códigos B78 vs B82: 0.164769647696477
Similitud fila 12 con códigos B99 vs T86: 0.0
Similitud fila 13 con códigos D11 vs D11: 1.0
Similitud fila 15 con códigos D72 vs D97: 0.0
Similitud fila 17 con códigos D82 vs D82: 1.0
Similitud fila 18 con códigos D83 vs D83: 1.0
Similitud fila 19 con códigos D83 vs D83: 1.0
Similitud fila 20 con códigos D87 vs D87: 1.0
Similitud fila 21 con códigos F70 vs F02: 0.38586664812289395
Similitud fila 22 con códigos F05 vs F94: 0.1268089538221613
Similitud fila 25 con códigos K75 vs K75: 1.0
Similitud fila 27 con códigos K79 vs K79: 1.0
Similitud fila 28 con códigos K83 vs K83: 1.0
Similitud fila 29 con códigos K29 vs K29: 1.0
Similitud fila 31 con códigos L76 vs L76: 1.0
Similitud fila 32 con códigos A97 vs A78: 0.0
Similitud fila 33 con c

In [19]:
df1 = pd.read_csv("datasetEtiquetado_INTER_1.csv")  
df3 = pd.read_csv("result_diag_graph_Snomed.csv", sep="|")
resultados_previos = pd.read_csv("resultados_similitud.csv")

nuevos_resultados = {}

for idx in resultados_previos["indice"].unique():
    row_etiquetado = df1[df1["indice"] == idx]
    rows_diag_graph = df3[df3["indice"] == idx]
    rows_differ_gt = rows_diag_graph[rows_diag_graph["level"].isin(["Differ GT", "Igual GT"])]
    rows_correcto = rows_diag_graph[rows_diag_graph["level"] == "correcto"]

    if row_etiquetado.empty:
        continue

    # df etiquetado cogemos snomed_gt
    snomed_raw_value = row_etiquetado.iloc[0]["snomed"]
    snomed_raw = str(snomed_raw_value).strip() if pd.notna(snomed_raw_value) else ""
    if snomed_raw.upper() == "NO CODIFICABLE":
        snomed_raw = ""
    snomed_codes = set([c.strip() for c in snomed_raw.split(";") if c.strip()] if snomed_raw else [])
    
    code_graph_set = set()
    similitudes_set = set()
    ciap_codes_set = set()
    similitudes_ciap_set = set()

    if not rows_differ_gt.empty:
        # df modelo cogemos snomed_code y ciap_code de las filas Differ GT
        unique_code_graphs = set(rows_differ_gt["code"].dropna().map(str.strip).unique())
        for info_text in rows_differ_gt["info"]:
            matches = re.findall(r"\b([A-Z]\d{2})\b", str(info_text))
            ciap_codes_set.update(matches)

        # sim entre snomed_gt y snomed_code
        for code_graph in unique_code_graphs:
            for snomed_code in snomed_codes:
                try:
                    sim = get_sim(snomed_code, code_graph)
                    sim_val = float(sim[0])
                    similitudes_set.add(f"{sim_val:.4f}")
                    code_graph_set.add(code_graph)
                    print(f"Similitud indice {idx} entre {snomed_code} y {code_graph}: {sim_val}")
                except Exception as e:
                    print(f"Error en indice {idx} entre {snomed_code} y {code_graph}: {e}")

    elif not rows_correcto.empty:
        for _, row in rows_correcto.iterrows():
            info_text = str(row["info"]).strip()
            match = re.search(r"(\d{6,})$", info_text)
            if match:
                code_graph = match.group(1)
                code_graph_set.add(match.group(1))

    nuevos_resultados[idx] = {
        "snomed_gt": snomed_raw,
        "snomed_code": ";".join(code_graph_set),
        "similitud_snomed": ";".join(similitudes_set),
        "ciap_code": ";".join(ciap_codes_set),
    }

for columna in ["snomed_gt", "snomed_code", "similitud_snomed", "ciap_code"]:
    resultados_previos[columna] = resultados_previos["indice"].map(lambda i: nuevos_resultados.get(i, {}).get(columna, ""))

resultados_previos.to_csv("resultados_similitud_actualizado.csv", index=False)
print("CSV actualizado guardado.")

Similitud indice 3 entre 131148009 y 12063002: 0.16682290236812364
Similitud indice 3 entre 131148009 y 26322001: 0.4368471381582998
Similitud indice 10 entre 703938007 y 5391000119108: 0.07245861602954913
Similitud indice 10 entre 47382004 y 5391000119108: 0.10548457515906619
Similitud indice 10 entre 703938007 y 399029005: 0.35296149037044283
Similitud indice 10 entre 47382004 y 399029005: 0.39796510057425194
Similitud indice 12 entre 414027002 y 362992004: 0.1734176439325175
Similitud indice 12 entre 106200001 y 362992004: 0.19813409407740043
Similitud indice 12 entre 414027002 y 232308006: 0.30401595023718136
Similitud indice 12 entre 106200001 y 232308006: 0.18448172403184118
Similitud indice 15 entre 3738000 y 57412004: 0.8366773344608247
Similitud indice 15 entre 3738000 y 37871000: 0.7333117859153346
Similitud indice 17 entre 105995000 y 238153009: 0.24689690588535362
Similitud indice 17 entre 105995000 y 18718003: 0.0701683655965605
Similitud indice 21 entre 243462001 y 982600

In [21]:
import pandas as pd
import concurrent.futures

result_df = pd.read_csv("resultados_similitud_actualizado.csv")

def calcular_similitud(row):

    code_gt_str = row.get("code_gt")
    ciap_code_str = row.get("ciap_code")

    if pd.isna(code_gt_str) or pd.isna(ciap_code_str):
        return ""

    code_gt = str(code_gt_str).strip()
    ciap_code = str(ciap_code_str).strip()

    if code_gt.upper() == "NO CODIFICABLE" or ciap_code == "":
        return ""

    codes_gt = [c.strip() for c in code_gt.split(";") if c.strip()]
    ciap_codes = [c.strip() for c in ciap_code.split(";") if c.strip()]

    if not codes_gt or not ciap_codes:
        return ""

    similitudes = []
    for code in codes_gt:
        for ciap in ciap_codes:
            try:
                sim = get_sim(code, ciap)
                sim_val = float(sim[0])
                similitudes.append(f"{sim_val:.4f}")
                print(f"Similitud entre {code} y {ciap}: {sim_val}")
            except Exception as e:
                print(f"Error calculando {code} vs {ciap}: {e}")
    return ";".join(similitudes)

# Multiproceso con ProcessPoolExecutor
with concurrent.futures.ProcessPoolExecutor() as executor:
    filas = [row for _, row in result_df.iterrows()]
    resultados = list(executor.map(calcular_similitud, filas))

result_df["similitud_ciap2"] = resultados

result_df.to_csv("resultados_similitud_actualizado_con_ciap2.csv", index=False)
print("CSV actualizado con columna similitud_ciap2.")

Similitud entre S74 y S74: 1.0
Similitud entre A10 y H05: 0.41500478125038476
Similitud entre S74 y S87: 0.10021034640111287
Similitud entre A10 y D16: 0.7288125727590219
Similitud entre S74 y S92: 0.6056232960906575
Similitud entre B99 y H82: 0.3756809579237406
Similitud entre S88 y S74: 0.5511657430091196
Similitud entre S88 y S87: 0.95
Similitud entre S88 y S92: 0.6516319198014383
Similitud entre F70 y F70: 1.0
Similitud entre F70 y F20: 0.700373095012828
Similitud entre D72 y L87: 0.361077677526268
Similitud entre F70 y F73: 0.786155682813625
Similitud entre D72 y R75: 0.25587807171192656
Similitud entre K29 y K74: 0.3114303959584575
Similitud entre A97 y A76: 0.0
Similitud entre A97 y R74: 0.0
Similitud entre F72 y F72: 1.0
Similitud entre F72 y F16: 0.7769312991001298
Similitud entre F72 y F72: 1.0
Similitud entre B99 y L99: 0.5125311792470931
Similitud entre F72 y F16: 0.7769312991001298
Similitud entre P98 y P23: 0.31335355661487035
Similitud entre S11 y S21: 0.2974312944707847

In [27]:
result_df = pd.read_csv("resultados_similitud_actualizado_con_ciap2.csv")

ciaps_cercanos_list = []
for _, row in result_df.iterrows():
    snomed_code_str = row.get("snomed_code", "")

    if pd.isna(snomed_code_str) or snomed_code_str.strip() == "":
        ciaps_cercanos_list.append("")
        continue

    all_ciaps = set()
    snomed_codes = [code.strip() for code in snomed_code_str.split(";") if code.strip()]

    for code in snomed_codes:
        try:
            result = get_top10(code, "CIAP1")
            ciap_codes = list(result)
            all_ciaps.update(ciap_codes)
            print(f"Para {code}: {ciap_codes}")
        except Exception as e:
            print(f"Error procesando {code}: {e}")

    ciaps_cercanos_list.append(";".join(sorted(all_ciaps)))

result_df["ciaps_cercanos"] = ciaps_cercanos_list
result_df.to_csv("resultados_con_ciaps_cercanos.csv", index=False)
print("CSV actualizado con columna ciaps_cercanos.")

Para 12063002: ['D16', 'D99', 'S71', 'Y99', 'Y06', 'L76', 'X82', 'D93', 'D95', 'D75']
Para 26322001: ['H05', 'H29', 'H99', 'H02', 'H13', 'A10', 'H04', 'H76', 'H80', 'A77']
Para 30746006: ['B03', 'B02', 'B70', 'B90', 'B71', 'N94', 'Y99', 'Y06', 'D29', 'X21']
Para 5391000119108: ['S88', 'S87', 'S92', 'S02', 'S99', 'S80', 'S75', 'S20', 'S98', 'H70']
Para 399029005: ['S74', 'D91', 'D89', 'S29', 'S08', 'S21', 'S89', 'D70', 'U70', 'A78']
Para 271737000: ['B82', 'B78', 'B84', 'B81', 'L18', 'L19', 'A90', 'D91', 'P29', 'A78']
Para 232308006: ['H82', 'S99', 'L88', 'N99', 'K71', 'L99', 'N94', 'W71', 'K99', 'B82']
Para 362992004: ['L99', 'S99', 'L88', 'N99', 'K71', 'N94', 'W71', 'K99', 'B82', 'S23']
Para 236071009: ['D99', 'D11', 'R83', 'R90', 'D70', 'D22', 'D92', 'D04', 'D98', 'D97']
Para 57412004: ['D72', 'A77', 'N70', 'A78', 'D97', 'D77', 'D71', 'A71', 'S71', 'N83']
Para 37871000: ['D72', 'R74', 'D88', 'L70', 'N70', 'L87', 'S76', 'R78', 'U70', 'R75']
Para 238153009: ['A78', 'L99', 'L18', 'L19',

In [29]:
import pandas as pd
import concurrent.futures

result_df = pd.read_csv("resultados_con_ciaps_cercanos.csv")

def calcular_similitud(row):

    code_gt_str = row.get("code_gt")
    ciap_code_str = row.get("ciaps_cercanos")

    if pd.isna(code_gt_str) or pd.isna(ciap_code_str):
        return ""

    code_gt = str(code_gt_str).strip()
    ciap_code = str(ciap_code_str).strip()

    if code_gt.upper() == "NO CODIFICABLE" or ciap_code == "":
        return ""

    codes_gt = [c.strip() for c in code_gt.split(";") if c.strip()]
    ciap_codes = [c.strip() for c in ciap_code.split(";") if c.strip()]

    if not codes_gt or not ciap_codes:
        return ""

    similitudes = []
    for code in codes_gt:
        for ciap in ciap_codes:
            try:
                sim = get_sim(code, ciap)
                sim_val = float(sim[0])
                similitudes.append(f"{sim_val:.4f}")
                print(f"Similitud entre {code} y {ciap}: {sim_val}")
            except Exception as e:
                print(f"Error calculando {code} vs {ciap}: {e}")
    return ";".join(similitudes)

# Multiproceso con ProcessPoolExecutor
with concurrent.futures.ProcessPoolExecutor() as executor:
    filas = [row for _, row in result_df.iterrows()]
    resultados = list(executor.map(calcular_similitud, filas))

result_df["similitud_ciaps_cercanos"] = resultados

result_df.to_csv("resultados_final.csv", index=False)
print("CSV actualizado.")

Similitud entre B02 y B02: 1.0
Similitud entre B02 y B03: 0.95
Similitud entre A10 y A10: 1.0
Similitud entre B02 y B70: 0.496053799091329
Similitud entre B02 y B71: 0.4716857796803444
Similitud entre B02 y B90: 0.3394579085340087
Similitud entre B02 y D29: 0.33465871994263036
Similitud entre B02 y N94: 0.38421766231673987
Similitud entre B02 y X21: 0.3131672378650703
Similitud entre B02 y Y06: 0.20795344248747422
Similitud entre B02 y Y99: 0.2933041214177128
Similitud entre B99 y B82: 0.4606037359296997
Similitud entre B78 y A78: 0.26893006291343574
Similitud entre B78 y A90: 0.26893006291343574
Similitud entre B78 y B78: 1.0
Similitud entre A10 y A77: 0.5256516831984024
Similitud entre B78 y B81: 0.6562203974362563
Similitud entre A10 y D16: 0.7288125727590219
Similitud entre B99 y H82: 0.3756809579237406
Similitud entre B78 y B82: 0.7436855857568678
Similitud entre A10 y D75: 0.13926824738438684
Similitud entre A10 y D93: 0.1978523534887555
Similitud entre B99 y K71: 0.3517179561482

In [13]:
result_df = pd.read_csv("resultados_final.csv")

def maxima_similitud(sim_str):
    if pd.isna(sim_str):
        return 1.0
    sim_str = str(sim_str).strip()
    if sim_str == "":
        return 1.0
    try:
        valores = [float(x) for x in str(sim_str).split(";") if x != "NA"]
        valores_positivos = [v for v in valores if v > 0]
        return max(valores_positivos) if valores_positivos else None
    except Exception:
        return None

result_df["max_similitud"] = result_df["similitud"].apply(maxima_similitud)
sim_values = result_df["max_similitud"].dropna()

# valores exactos 1.0 y distintos de 1.0
num_unos = (sim_values == 1.0).sum()
num_entre_07_y_menor_1 = ((sim_values >= 0.7) & (sim_values < 1.0)).sum()
num_menor_07 = (sim_values < 0.7).sum()
total = len(sim_values)

# porcentajes
porcentaje_unos = (num_unos / total) * 100 if total > 0 else 0
porcentaje_entre_07_y_menor_1 = (num_entre_07_y_menor_1 / total) * 100 if total > 0 else 0
porcentaje_menor_07 = (num_menor_07 / total) * 100 if total > 0 else 0

# media de los distintos de 1.0
media_distinto_uno = sim_values[sim_values != 1.0].mean()

print(f"ciap_gt vs diagCD")
print(f"-----------------")
print(f"Número de valores == 1.0: {num_unos} ({porcentaje_unos:.2f}%)")
print(f"Número de valores >= 0.7 y < 1.0: {num_entre_07_y_menor_1} ({porcentaje_entre_07_y_menor_1:.2f}%)")
print(f"Número de valores < 0.7: {num_menor_07} ({porcentaje_menor_07:.2f}%)")
print(f"Media de las similitudes distintas de 1.0: {media_distinto_uno:.4f}")



result_df["max_similitud_snomed"] = result_df["similitud_snomed"].apply(maxima_similitud)
result_df["max_similitud_ciaps_cercanos"] = result_df["similitud_ciaps_cercanos"].apply(maxima_similitud)
total_filas = len(result_df)

num_unos_snomed = (result_df["max_similitud_snomed"] == 1.0).sum()
porc_unos_snomed = (num_unos_snomed / total_filas) * 100

casos_ciap_1_sino_snomed = result_df[
    (result_df["max_similitud_snomed"] != 1.0) & (result_df["max_similitud_ciaps_cercanos"] == 1.0)
]
num_snomed_no_1_ciap_si_1 = len(casos_ciap_1_sino_snomed)
porc_snomed_no_1_ciap_si_1 = (num_snomed_no_1_ciap_si_1 / total_filas) * 100

resto = result_df[
    (result_df["max_similitud_snomed"] != 1.0) & 
    (result_df["max_similitud_ciaps_cercanos"] != 1.0)
].copy()
sim_snomed_resto = resto["max_similitud_snomed"].dropna()
sim_ciaps_resto = resto["max_similitud_ciaps_cercanos"].dropna()
sim_maximo_resto = resto[["max_similitud_snomed", "max_similitud_ciaps_cercanos"]].max(axis=1).dropna()

def resumen_stats(nombre, serie):
    num_entre_07_y_menor_1 = ((serie >= 0.7) & (serie < 1.0)).sum()
    num_menor_07 = (serie < 0.7).sum()
    media = serie[serie != 1.0].mean()
    porcentaje_entre_07_y_menor_1 = (num_entre_07_y_menor_1 / total_filas) * 100
    porcentaje_menor_07 = (num_menor_07 / total_filas) * 100

    print(f"\n{nombre}:")
    print(f"Número de valores >= 0.7 y < 1.0: {num_entre_07_y_menor_1} ({porcentaje_entre_07_y_menor_1:.2f}%)")
    print(f"Número de valores < 0.7: {num_menor_07} ({porcentaje_menor_07:.2f}%)")
    print(f"Media de las similitudes distintas de 1.0: {media:.4f}")

print("\nsnomed_code y ciaps_cercanos")
print("----------------------------")
print(f"Número de valores == 1.0 snomed_gt vs snomed_code: {num_unos_snomed} ({porc_unos_snomed:.2f}%)")
print(f"Número de valores == 1.0 ciap_gt vs ciaps_cercanos extras: {num_snomed_no_1_ciap_si_1} ({porc_snomed_no_1_ciap_si_1:.2f}%)")
print(f"Total número de valores == 1.0: {num_unos_snomed+num_snomed_no_1_ciap_si_1} ({porc_unos_snomed+porc_snomed_no_1_ciap_si_1:.2f}%)")
resumen_stats("Solo SNOMED del resto", sim_snomed_resto)
resumen_stats("Solo CIAPs cercanos del resto", sim_ciaps_resto)
resumen_stats("Máximo del resto", sim_maximo_resto)

ciap_gt vs diagCD
-----------------
Número de valores == 1.0: 1802 (69.41%)
Número de valores >= 0.7 y < 1.0: 197 (7.59%)
Número de valores < 0.7: 597 (23.00%)
Media de las similitudes distintas de 1.0: 0.4822

snomed_code y ciaps_cercanos
----------------------------
Número de valores == 1.0 snomed_gt vs snomed_code: 1651 (61.56%)
Número de valores == 1.0 ciap_gt vs ciaps_cercanos extras: 477 (17.79%)
Total número de valores == 1.0: 2128 (79.34%)

Solo SNOMED del resto:
Número de valores >= 0.7 y < 1.0: 10 (0.37%)
Número de valores < 0.7: 544 (20.28%)
Media de las similitudes distintas de 1.0: 0.2535

Solo CIAPs cercanos del resto:
Número de valores >= 0.7 y < 1.0: 147 (5.48%)
Número de valores < 0.7: 407 (15.18%)
Media de las similitudes distintas de 1.0: 0.5459

Máximo del resto:
Número de valores >= 0.7 y < 1.0: 151 (5.63%)
Número de valores < 0.7: 403 (15.03%)
Media de las similitudes distintas de 1.0: 0.5484


In [15]:
df = pd.read_csv("resultados_final.csv")

ciap_max_list = []
for _, row in df.iterrows():
    ciaps = str(row.get("ciaps_cercanos", "")).split(";")
    sims_raw = str(row.get("similitud_ciaps_cercanos", "")).split(";")
    
    try:
        sims = [float(s) for s in sims_raw]
    except:
        ciap_max_list.append("")
        continue

    if len(ciaps) * len(str(row.get("code_gt", "")).split(";")) == len(sims) and sims:
        max_index = sims.index(max(sims))
        ciap_max_list.append(ciaps[max_index % len(ciaps)])
    elif len(ciaps) == len(sims) and sims:
        max_index = sims.index(max(sims))
        ciap_max_list.append(ciaps[max_index])
    else:
        ciap_max_list.append("")

df["ciap_max_similitud"] = ciap_max_list

df["coincide_con_code_gt"] = df.apply(
    lambda row: row["ciap_max_similitud"] in str(row["code_gt"]).split(";"), axis=1
)

num_coinciden = df["coincide_con_code_gt"].sum()
num_no_coinciden = (~df["coincide_con_code_gt"]).sum()

print(f"Número de filas donde ciap_max_similitud coincide con alguno en code_gt: {num_coinciden}")
print(f"Número de filas donde no coincide: {num_no_coinciden}")

df.to_csv("resultados_final_con_ciap_max.csv", index=False)
print("CSV actualizado con columna 'ciap_max_similitud' guardado como 'resultados_final_con_ciap_max.csv'.")

Número de filas donde ciap_max_similitud coincide con alguno en code_gt: 1898
Número de filas donde no coincide: 784
CSV actualizado con columna 'ciap_max_similitud' guardado como 'resultados_final_con_ciap_max.csv'.
